In [1]:
from pipeline import process_dir

dir = r"C:\Users\Levi\Desktop\localized-clock-roi-ds-raw-files\trimmed-videos"
process_dir(dir)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Find ROIs for video at C:\Users\Levi\Desktop\localized-clock-roi-ds-raw-files\trimmed-videos\broadcast_1.mp4


  0%|          | 0/601 [00:00<?, ?it/s]

# Semi-Supervised DB Expansion
***

In [ ]:
# for row in predictions:
#     # Convert row values to strings and join them with a space
#     row_string = ' '.join([f'{value:.4f}' for value in row])
#     print(row_string)